<a href="https://colab.research.google.com/github/shabd01/Image-Classification-Model/blob/main/notebooke40fd243db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-classification-2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F35049%2F3311124%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T033416Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D219c8605d4aaa4bdd7e909e87419d06ac8807f3bf61744e1cf6d43a9d1c89fc5db96bc4549281fc7f658981a64277d8f0f02bef0af0cda28b4e9e54bfa92cd510bff2696f105c954c93890aa17a02d85df14849e3ca3cb2748e7cf89cfce9b0c7c497265d25a73824cd2db616e23ef2d1af590aa6232209b0b99bcf76c585f8182fcb76b0d4f767120a3b18320d344c408fb09c198a1fb6ba91d50101f24ae937af0927d3a6db9189be71dfd677423a0ad7dd97fbfdc62161ad6bd9223078d559010dde0434c77c73685776947c5ec072ed535c773f3b422d168dbee44439ee048d18b47af05e26b6eb299ae18a197b36567f69ecfc55d656d2f043b119fb471'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import os.path
import keras_tuner as kt
import cv2


In [ ]:
from PIL import Image
import os
import PIL
import glob

In [ ]:
train = pd.read_csv('../input/image-classification-2/train_label.csv')

In [ ]:
train_image = []
for i in tqdm(range(10)):
    img = image.load_img('../input/image-classification-2/train/'+'Image_00000' + str(i) +'.jpg', target_size= None, grayscale=False)
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    train_image.append(img)
for i in tqdm(range(10,100)):
    img = image.load_img('../input/image-classification-2/train/'+'Image_0000' + str(i) +'.jpg', target_size= None, grayscale=False)
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    train_image.append(img)
for i in tqdm(range(100,1000)):
    img = image.load_img('../input/image-classification-2/train/'+'Image_000' + str(i) +'.jpg', target_size= None, grayscale=False)
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    train_image.append(img)
for i in tqdm(range(1000,10000)):
    if os.path.exists('../input/image-classification-2/train/'+'Image_00' + str(i) +'.jpg'):
       img = image.load_img('../input/image-classification-2/train/'+'Image_00' + str(i) +'.jpg', target_size= None, grayscale=False)
    else :
       continue
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    train_image.append(img)
for i in tqdm(range(10000,31838)):
    if os.path.exists('../input/image-classification-2/train/'+'Image_0' + str(i) +'.jpg'):
      img = image.load_img('../input/image-classification-2/train/'+'Image_0' + str(i) +'.jpg', target_size= None, grayscale=False)
    else :
       continue
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
y=train['label'].values
y = to_categorical(y)
len(y)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.4, shear_range=0.5, zoom_range=0.5,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(60,60,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=aug.flow(X_train, y_train, batch_size=64),
                    epochs = 15,
                    validation_data = (X_test, y_test))

In [ ]:
prediction = []
classes = [ "0","1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
for i in tqdm(range(10)):
    img = image.load_img('../input/image-classification-2/test/'+'Image_test_00000' + str(i) +'.jpg', target_size= None, grayscale=False)
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    proba = model.predict(img.reshape(1,60,60,3))
    top_1 = np.argsort(proba[0])[:-4:-1]
    prediction.append("{}".format(classes[top_1[0]]))
for i in tqdm(range(10,100)):
    img = image.load_img('../input/image-classification-2/test/'+'Image_test_0000' + str(i) +'.jpg', target_size= None, grayscale=False)
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    proba = model.predict(img.reshape(1,60,60,3))
    top_1 = np.argsort(proba[0])[:-4:-1]
    prediction.append("{}".format(classes[top_1[0]]))
for i in tqdm(range(100,1000)):
    img = image.load_img('../input/image-classification-2/test/'+'Image_test_000' + str(i) +'.jpg', target_size=None, grayscale=False)
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    proba = model.predict(img.reshape(1,60,60,3))
    top_1 = np.argsort(proba[0])[:-4:-1]
    prediction.append("{}".format(classes[top_1[0]]))
for i in tqdm(range(1000,10000)):
    if os.path.exists('../input/image-classification-2/test/'+'Image_test_00' + str(i) +'.jpg'):
       img = image.load_img('../input/image-classification-2/test/'+'Image_test_00' + str(i) +'.jpg', target_size= None, grayscale=False)
    else :
       continue
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    proba = model.predict(img.reshape(1,60,60,3))
    top_1 = np.argsort(proba[0])[:-4:-1]
    prediction.append("{}".format(classes[top_1[0]]))
for i in tqdm(range(10000,18229)):
    if os.path.exists('../input/image-classification-2/test/'+'Image_test_0' + str(i) +'.jpg'):
      img = image.load_img('../input/image-classification-2/test/'+'Image_test_0' + str(i) +'.jpg', target_size= None, grayscale=False)
    else :
       continue
    resized_image = img.resize((60,60))
    img = image.img_to_array(resized_image)
    img = img/255
    proba = model.predict(img.reshape(1,60,60,3))
    top_1 = np.argsort(proba[0])[:-4:-1]
    prediction.append("{}".format(classes[top_1[0]]))


In [ ]:
sample = pd.read_csv('../input/image-classification-2/sample_submission.csv')

In [ ]:

sample['label'] = prediction
sample.to_csv('submission.csv', header=True, index=False)